In [16]:
!python train.py

Namespace(batch_size=128, cuda=True, decoder='mlp', decoder_dropout=0.0, decoder_hidden=256, dims=4, dynamic_graph=False, edge_types=2, encoder='mlp', encoder_dropout=0.0, encoder_hidden=256, epochs=5, factor=True, gamma=0.5, hard=False, load_folder='', lr=0.0005, lr_decay=200, no_cuda=False, no_factor=False, num_atoms=5, prediction_steps=10, prior=False, save_folder='logs', seed=42, skip_first=False, suffix='_springs5', temp=0.5, timesteps=49, var=5e-05)
/home/jovyan/NRI-master/modules.py:27: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(m.weight.data)
Using factor graph MLP encoder.
/home/jovyan/NRI-master/modules.py:110: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(m.weight.data)
Using learned interaction net decoder.
/home/jovyan/.imgenv-baseline-0/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_schedul

In [23]:
from modules import *

In [26]:
#default
timesteps = 49
dims = 4
encoder_hidden = 256
edge_types = 2
encoder_dropout = 0.0
factor = False 
decoder_hidden = 256
decoder_dropout = 0.0
skip_first = False

encoder = MLPEncoder(timesteps * dims, encoder_hidden,
                         edge_types,
                         encoder_dropout, factor)
decoder = MLPDecoder(n_in_node=dims,
                         edge_types=edge_types,
                         msg_hid=decoder_hidden,
                         msg_out=decoder_hidden,
                         n_hid=decoder_hidden,
                         do_prob=decoder_dropout,
                         skip_first=skip_first)

Using MLP encoder.
Using learned interaction net decoder.


/home/jovyan/NRI-master/modules.py:27: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(m.weight.data)
/home/jovyan/NRI-master/modules.py:110: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(m.weight.data)


In [29]:
import os

encoder_file = os.path.join('logs/exp2023-07-24T10:27:08.443431', 'encoder.pt')
decoder_file = os.path.join('logs/exp2023-07-24T10:27:08.443431', 'decoder.pt')

In [21]:
train = np.load('data/train_small.npy')

In [22]:
train.shape[3]

52

In [2]:
import numpy as np
import torch
from torch.utils.data.dataset import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.autograd import Variable

In [3]:
def my_load_data(batch_size=1, suffix=''):
    
    train = np.load('data/train_small.npy')
    valid = np.load('data/test_small.npy')


    # [num_samples, num_timesteps, num_dims, num_atoms]
    num_atoms = train.shape[3]

    loc_max = -1
    loc_min = -1
    vel_max = -1
    vel_min = -1

    # Normalize to [-1, 1]
    #loc_train = (loc_train - loc_min) * 2 / (loc_max - loc_min) - 1
    #vel_train = (vel_train - vel_min) * 2 / (vel_max - vel_min) - 1

    #loc_valid = (loc_valid - loc_min) * 2 / (loc_max - loc_min) - 1
    #vel_valid = (vel_valid - vel_min) * 2 / (vel_max - vel_min) - 1

    #loc_test = (loc_test - loc_min) * 2 / (loc_max - loc_min) - 1
    #vel_test = (vel_test - vel_min) * 2 / (vel_max - vel_min) - 1

    # Reshape to: [num_sims, num_atoms, num_timesteps, num_dims]
    feat_train = np.transpose(train, [0, 3, 1, 2])
    feat_valid = np.transpose(valid, [0, 3, 1, 2])

    feat_train = torch.FloatTensor(feat_train)
    feat_valid = torch.FloatTensor(feat_valid)

    # Exclude self edges
    off_diag_idx = np.ravel_multi_index(
        np.where(np.ones((num_atoms, num_atoms)) - np.eye(num_atoms)),
        [num_atoms, num_atoms])

    edges_train = feat_train # мусор
    edges_valid = feat_valid # мусор
    
    train_data = TensorDataset(feat_train, edges_train)
    valid_data = TensorDataset(feat_valid, edges_valid)

    train_data_loader = DataLoader(train_data, batch_size=batch_size)
    valid_data_loader = DataLoader(valid_data, batch_size=batch_size)
    
    test_data_loader = DataLoader(valid_data, batch_size=batch_size) # фиктивная строка

    return train_data_loader, valid_data_loader, test_data_loader, loc_max, loc_min, vel_max, vel_min

In [5]:
train_loader, valid_loader, test_loader, loc_max, loc_min, vel_max, vel_min = my_load_data(
    50)

In [6]:
len(train_loader)

99

In [7]:
len(valid_loader)

20

In [ ]:
!python train-my.py

Namespace(batch_size=50, cuda=True, decoder='mlp', decoder_dropout=0.0, decoder_hidden=128, dims=1, dynamic_graph=False, edge_types=2, encoder='mlp', encoder_dropout=0.0, encoder_hidden=128, epochs=100, factor=True, gamma=0.5, hard=False, load_folder='logs/exp2023-07-26T16:27:04.767683', lr=0.0005, lr_decay=200, no_cuda=False, no_factor=False, num_atoms=52, prediction_steps=10, prior=False, save_folder='logs', seed=42, skip_first=False, suffix='_springs5', temp=0.5, timesteps=30, var=5e-05)
/home/jovyan/NRI-master/modules.py:27: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(m.weight.data)
Using factor graph MLP encoder.
/home/jovyan/NRI-master/modules.py:110: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(m.weight.data)
Using learned interaction net decoder.
/home/jovyan/.imgenv-baseline-0/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserW

In [14]:
print('ok')

ok


In [ ]:
!python train-my.py

Namespace(batch_size=50, cuda=True, decoder='mlp', decoder_dropout=0.0, decoder_hidden=128, dims=1, dynamic_graph=False, edge_types=2, encoder='mlp', encoder_dropout=0.0, encoder_hidden=128, epochs=100, factor=True, gamma=0.5, hard=False, load_folder='', lr=0.0005, lr_decay=200, no_cuda=False, no_factor=False, num_atoms=52, prediction_steps=10, prior=False, save_folder='logs', seed=42, skip_first=False, suffix='_springs5', temp=0.5, timesteps=30, var=5e-05)
/home/jovyan/NRI-master/modules.py:27: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(m.weight.data)
Using factor graph MLP encoder.
/home/jovyan/NRI-master/modules.py:110: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(m.weight.data)
Using learned interaction net decoder.
/home/jovyan/.imgenv-baseline-0/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_sched

In [15]:

import pickle

with open('logs/exp2023-07-27T01:09:21.423069/metadata.pkl', 'rb') as f:
    data = pickle.load(f)
data        

{'args': Namespace(batch_size=50, cuda=True, decoder='mlp', decoder_dropout=0.0, decoder_hidden=128, dims=1, dynamic_graph=False, edge_types=2, encoder='mlp', encoder_dropout=0.0, encoder_hidden=128, epochs=100, factor=True, gamma=0.5, hard=False, load_folder='logs/exp2023-07-26T16:27:04.767683', lr=0.0005, lr_decay=200, no_cuda=False, no_factor=False, num_atoms=52, prediction_steps=10, prior=False, save_folder='logs', seed=42, skip_first=False, suffix='_springs5', temp=0.5, timesteps=30, var=5e-05)}

In [19]:
from collections import OrderedDict

In [20]:
new_dict = OrderedDict([(key.replace('model.', ''), value) for key, value in old_dict.items()])

NameError: name 'old_dict' is not defined